# Init

In [13]:
import sys
import os
import json
import time
import re

from tqdm import tqdm

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data
import embedding

In [14]:
version = 'v2.7.1'

In [15]:
system_prompt = ""

prompt_dir = os.path.join(cwd, 'prompt')
fname = f'table_finder-stage_1-{version}.md'
prompt_fpath = os.path.join(prompt_dir, fname)

with open(prompt_fpath, 'r') as f:
    prompt_template = ''.join(f.readlines())


def parse_database_and_table(query: str) -> dict:
    """
    Parse the given SQL query to return the database and table names in a dictionary.
    
    Args:
    - query (str): The SQL query to parse.
    
    Returns:
    - dict: A dictionary with 'database' and 'table' keys.
    """

    pattern = r'FROM\s+([a-zA-Z0-9_]+)\.([a-zA-Z0-9_]+)'
    match = re.search(pattern, query, re.IGNORECASE)
    
    if match:
        database = match.group(1)
        table = match.group(2)
        return {'database': database, 'table': table}
    
    return {}


def make_prompt(query: str, ner: dict) -> str:

    """
    ner_res: content from the stage_1
    """

    prompt = prompt_template + query

    # ner_result is None
    if not ner['result']:
        return prompt

    ner_content = {}
    # assume there is only one NER result
    ner_content.update(ner['result'][0])

    sql_res = ner['sql']

    for k, v in sql_res.items():
        for j in v:
            if not j['result']:
                continue

            sql_query = j['query']
            # add database and table
            ner_content.update(parse_database_and_table(sql_query))
            # get market
            if ner_content['table'] == 'US_SecuMain':
                ner_content['market'] = 'US'
            elif ner_content['table'] == 'HK_SecuMain':
                ner_content['market'] = 'HK'
            else:
                ner_content['market'] = 'CN'
            ner_content['data_from_table'] = j['result']

    # add NER result

    ner_str = f"\n\n### **Name Entity Recognition Result**\n```json\n{json.dumps(ner_content, ensure_ascii=False,indent=2)}\n```"

    prompt += ner_str
    
    return prompt

In [16]:
question_path = os.path.join(cwd, 'answer_tmp' + os.sep + 'stage_1-glm_4_plus-ner-v2.0.0-sql-HF-Post.json')

questions = parse_data.read_json(question_path)

# GLM-4-Plus

In [4]:
model = 'glm_4_plus'

## Test

In [ ]:
query = make_prompt(questions[12]['team'][0]['question'], questions[12]['ner']['stage_1'])

history = []

start_time = time.time()
message = chat.create_message(query, history=history, system_prompt=system_prompt, temperature=0.7, top_p=0.9, response_format='text')
end_time = time.time()

execution_time = end_time - start_time
usage = chat.get_token_usage(message, True)
content = chat.get_content(message, True)
history = chat.build_history(history, message=message)

In [6]:
t = questions[12]
t['table_finder'] = {}
t['table_finder']['stage_1']= [json.loads(content.strip('`json'))]
t['token_usage'] = {}
t['token_usage']['table_finder-stage_1'] = [usage]
t['time_usage'] = {}
t['time_usage']['table_finder-stage_1'] = [f"{execution_time:.2f}s"]
t = [t]

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-table_finder-test-{version}.json')
parse_data.write_json(t, saved_path)

## ALL

In [ ]:
answers = []

for question in tqdm(questions[:]):
    try:
        # the first question
        query = make_prompt(question['team'][0]['question'], question['ner']['stage_1'])

        history = []

        start_time = time.time()
        message = chat.create_message(query, history=history, system_prompt=system_prompt, temperature=0.7, top_p=0.9, response_format='text')
        end_time = time.time()

        execution_time = end_time - start_time
        usage = chat.get_token_usage(message, False)
        content = chat.get_content(message, False)

        res = question
        res['table_finder'] = {}
        res['table_finder']['stage_1']= [json.loads(content.strip('`json'))]
        res['token_usage']['table_finder-stage_1'] = [usage]
        res['time_usage']['table_finder-stage_1'] = [f"{execution_time:.2f}s"]

        answers.append(res)
    except:
        print(question['tid'])

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-table_finder-{version}.json')
parse_data.write_json(answers, saved_path)

# Deepseek-chat

In [17]:
model = 'deepseek_v3'

In [18]:
deepseek_api = 'sk-ba0f5eed3bea4fa6be16eb33b139c684'

## Test

In [19]:
from openai import OpenAI

query = make_prompt(questions[0]['team'][0]['question'], questions[0]['ner']['stage_1'])

client = OpenAI(api_key= deepseek_api, base_url="https://api.deepseek.com")

start_time = time.time()
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "user", "content": query},
    ],
    stream=False,
    top_p=0.7,
    temperature=0.9
)
end_time = time.time()

response = json.loads(response.to_json())
content = response['choices'][0]['message']['content']

content = content.strip('`json')
usage = response['usage']
execution_time = end_time - start_time

In [20]:
t = questions[0]
t['table_finder'] = {}
t['table_finder']['stage_1']= [json.loads(content.strip('`json'))]
t['token_usage'] = {}
t['token_usage']['table_finder-stage_1'] = [usage]
t['time_usage'] = {}
t['time_usage']['table_finder-stage_1'] = [f"{execution_time:.2f}s"]
t = [t]

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-table_finder-test-{version}.json')
parse_data.write_json(t, saved_path)

## ALL

In [ ]:
answers = []

for question in tqdm(questions[:]):
    
    query = make_prompt(question['team'][0]['question'], question['ner']['stage_1'])

    start_time = time.time()
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": query},
        ],
        stream=False,
        top_p=0.7,
        temperature=0.9
    )
    end_time = time.time()

    response = json.loads(response.to_json())
    content = response['choices'][0]['message']['content']
    content = content.strip('`json')
    usage = response['usage']
    execution_time = end_time - start_time

    res = question
    res['table_finder'] = {}
    res['table_finder']['stage_1']= [json.loads(content.strip('`json'))]
    res['token_usage']['table_finder-stage_1'] = [usage]
    res['time_usage']['table_finder-stage_1'] = [f"{execution_time:.2f}s"]

    answers.append(res)

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-table_finder-{version}.json')
parse_data.write_json(answers, saved_path)

# Compare Results

Compare the stage 1 results => find the differences => get the correct answer.

In [ ]:
import os
import json
import copy

# Path to the folder containing model answer files
dir_path = os.path.join(cwd, 'answer_tmp')

# List of model names
models = ['deepseek_v3', 'glm_4_plus']

# Create a dictionary of file paths for each model's JSON file
model_files = {model: os.path.join(dir_path, f"stage_1-{model}-table_finder-{version}.json") for model in models}

# Dictionary to store the data of each model
model_data = {}

# Read the JSON data for each model
for model, file_path in model_files.items():
    with open(file_path, 'r', encoding='utf-8') as f:
        model_data[model] = json.load(f)

# sort data
for key in model_data:
    model_data[key] = sorted(model_data[key], key=lambda x: x['tid'])

# Dictionary to store the data_source for each question id across different models
data_sources = {}

# Traverse through each model's data to extract the data_source for each question id
for model in models:
    for entry in copy.deepcopy(model_data[model]):
        tmp = entry['table_finder']['stage_1'][0]['data_source']
        data_sources.setdefault(entry['tid'].replace(' ', ''), {}).update({model: tmp})

# Compare the data_source for each question id across models
for question_id, sources in data_sources.items():
    # Check if the data_source is consistent across models
    # Convert each model's data_source to a JSON string (to handle the dictionary comparison)
    results = [v for k, v in sources.items()]
    if results[0] == results[1]:
        pass
    else:
        print(f"Question ID: {question_id}")

        q_id = int(question_id.split('-')[-1]) - 1
        
        # Calculate the maximum length of model names to align the output
        max_model_length = max(len(model) for model in models)
        # Print the data_source for each model, with aligned output
        for model in models:
            # Left-align model names with the calculated maximum length
            print(f"{model.ljust(max_model_length)}:")
            print('```')
            for d in model_data[model]:
                if d['tid'] == question_id:
                    print(json.dumps(d['table_finder']['stage_1'], indent=2, ensure_ascii=False))
            print('```')
        
        print("")